# Aufbau der Notebooks [Philipp]

- Multivalue bei Notebooks angucken
- Wie installiere ich den "scheiß"
- Widgets: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html


In [74]:
from enum import Enum
import ipywidgets as widgets
from pandas import read_csv
import os
import numpy as np
import time

In [55]:
# Globals

PATH_RAW_DATA = "./data/raw/"
PATH_FEATURE_DATA = "./data/"
PATH_EXPLORATION_DATA = "./exploration/"
PATH_MODEL = "./models/"
DATA_SOURCE_KIDAQ = ["TEST_NAME", "TEST_TYPE", "RPM", "FLOW_RATE", "P1", "P2"]
RAW_DATA_TYPE = ["KIDAQ", "VIB"]

DATA_SOURCE_VID = [
    "TEST_NAME",
    "TEST_TYPE",
    "RPM",
    "FLOW_RATE",
    "S1",
    "S2",
    "S3",
    "S4",
    "S5",
    "S6",
    "S7",
    "S8",
]
FEATURE = [
    "STD",
    "RANGE",
    "IQR",
    "MEAN_MEDIAN",
    "FFT",
]
OPERATING_POINT_FREQ = [725, 1450, 2175, 2900]
OPERATING_POINT_FLOW_RATE = [0, 25, 50, 75, 100]

LEARNER = Enum("LEARNER", ["DNN", "DT"])

DEFAULT_LEARNER = LEARNER.DNN
DEFAULT_RAW_DATA_TYPE = RAW_DATA_TYPE[1]
DEFAULT_RAW_DATA = DATA_SOURCE_KIDAQ

DEFAULT_CLASS_LABEL = "TEST_TYPE"

tags = widgets.TagsInput(
    value=DEFAULT_RAW_DATA, allowed_tags=DEFAULT_RAW_DATA, allow_duplicates=False
)
display(tags)

TagsInput(value=['TEST_NAME', 'TEST_TYPE', 'RPM', 'FLOW_RATE', 'P1', 'P2'], allow_duplicates=False, allowed_ta…

## 1. Aufgabe und Daten erklären/beschreiben [Philipp]


## 3. Preprocessing [Valerij]

Aufgeteilt nach KIDAQ und VIB (separat um es einfach zu halten)

- Auswahl der Fenstergröße in Millisekunden
- Auswahl der Abtastrate
- Multi-Selektion der Aggregation (avg, mean, std, ...)
- Frequenzanalyse
- Fourier-Transformation
- Fenstergröße nach Frequenzbereichen

### 3.1 Vorbereitung der Tainings- und Testdaten

Multi-Selektion für:

- Features
- Betriebspunkte (RPM/FLOWRATE)
- Klassifikationsarten (Szenario / Testdurchlauf)


## 4. Deskriptive/Explorative Datenanalyse [Philipp]

- Plots
- Beschreibung der Plots


## 5. Machine Learning [Kevin]

Multi-Selektion für:

- Auswahl der Featuredateien (Train/Testdaten)
- Auswahl des Learners
- Konfiguration der Hyperparameter
- Live-Validation des Models mit vorausgewählten Testdaten (Random-Search, ...)


In [47]:
# SELECT TRAINING AND TEST DATA
featureDataDir = list(filter(lambda x: os.path.isfile(os.path.join(PATH_FEATURE_DATA, x)) and DEFAULT_RAW_DATA_TYPE in x.upper(), os.listdir(PATH_FEATURE_DATA)))

trainingFileDropdown = widgets.Dropdown(description="training features")
trainingFileDropdown.options = featureDataDir
selectedTrainingFile = None
def onTrainigFileChange(change):
    global selectedTrainingFile
    selectedTrainingFile = change['new']
trainingFileDropdown.observe(onTrainigFileChange, names='value')
display(trainingFileDropdown)

testFileDropdown = widgets.Dropdown(description="test features")
testFileDropdown.options = featureDataDir
selectedTestFile = None
def onTestFileChange(change):
    global selectedTestFile
    selectedTestFile = change['new']
testFileDropdown.observe(onTestFileChange, names='value')
display(testFileDropdown)


Dropdown(description='training features', options=('vib.setup1.csv', 'vib.setup2.csv'), value=None)

Dropdown(description='test features', options=('vib.setup1.csv', 'vib.setup2.csv'), value=None)

In [61]:
# SET ML CONFIG
FEATURE_TRAIN_FILE = PATH_FEATURE_DATA + selectedTrainingFile
FEATURE_TEST_FILE = PATH_FEATURE_DATA + selectedTestFile

DEFAULT_OPERATING_POINTS = OPERATING_POINT_FREQ + OPERATING_POINT_FLOW_RATE


# DNN
DNN_EXPLORATION_TARGET_VAL_ACCURACY = 0.9
DNN_EXPLORATION_MAX_ITER = 1
DNN_EXPLORATION_HIDDEN_LAYERS_MIN = 2
DNN_EXPLORATION_HIDDEN_LAYERS_MAX = 4
DNN_EXPLORATION_NEURONS_MIN = 8
DNN_EXPLORATION_NEURONS_MAX = 64


DNN_EARLY_STOPPING_PATIENCE = 50
DNN_VERBOSE = 0
DNN_EPOCHS = 100
DNN_BATCH_SIZE = 32
DNN_BATCH_NORMALIZATION = True


# DT
DT_MAX_DEPTH = 6
DT_NUM_ESTIMATORS = 100

In [59]:
# LOAD DATA
train_data = read_csv(FEATURE_TRAIN_FILE, header=None, delimiter=";").values
test_data = read_csv(FEATURE_TEST_FILE, header=None, delimiter=";").values

train_x, train_y = train_data[:, 2:].astype('float32'), train_data[:, 1:2]
test_x, test_y = test_data[:, 2:].astype('float32'), test_data[:, 1:2]

In [78]:
# Train Decision Tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier


labelEncoder = LabelEncoder()
labelEncoder = labelEncoder.fit(np.ravel(train_y))
label_encoded_train_y = labelEncoder.transform(np.ravel(train_y))
label_encoded_test_y = labelEncoder.transform(np.ravel(test_y))


xgb = XGBClassifier(
    tree_method="hist",
    enable_categorical=True,
    max_depth=DT_MAX_DEPTH,
    n_estimators=DT_NUM_ESTIMATORS,
)
# fit model
xgb.fit(train_x, label_encoded_train_y, eval_set=[(test_x, label_encoded_test_y)])


preds = xgb.predict(test_x)
accuracy = accuracy_score(label_encoded_test_y, preds)

XGBClassifier.save_model(xgb, PATH_MODEL+"dt."+str(round(time.time())))

[0]	validation_0-mlogloss:1.27518
[1]	validation_0-mlogloss:1.28511
[2]	validation_0-mlogloss:1.21706
[3]	validation_0-mlogloss:1.21787
[4]	validation_0-mlogloss:1.24143
[5]	validation_0-mlogloss:1.27339
[6]	validation_0-mlogloss:1.32524
[7]	validation_0-mlogloss:1.42268
[8]	validation_0-mlogloss:1.52952
[9]	validation_0-mlogloss:1.59830
[10]	validation_0-mlogloss:1.63602
[11]	validation_0-mlogloss:1.74402
[12]	validation_0-mlogloss:1.85493
[13]	validation_0-mlogloss:1.96412
[14]	validation_0-mlogloss:2.07208
[15]	validation_0-mlogloss:2.17972
[16]	validation_0-mlogloss:2.27525
[17]	validation_0-mlogloss:2.33593
[18]	validation_0-mlogloss:2.40999
[19]	validation_0-mlogloss:2.50217
[20]	validation_0-mlogloss:2.58418
[21]	validation_0-mlogloss:2.61032
[22]	validation_0-mlogloss:2.69617
[23]	validation_0-mlogloss:2.71945
[24]	validation_0-mlogloss:2.76588
[25]	validation_0-mlogloss:2.81289
[26]	validation_0-mlogloss:2.87667
[27]	validation_0-mlogloss:2.92785
[28]	validation_0-mlogloss:2.9

In [77]:
# Train Neural Network
import random
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

onehotencoder = OneHotEncoder()
onehotencoder = onehotencoder.fit(train_y)
onehot_encoded_train_y = onehotencoder.transform(train_y).toarray()
onehot_encoded_test_y = onehotencoder.transform(test_y).toarray()

exploration_results = []


n_features = train_x.shape[1]
categories = len(onehot_encoded_train_y[0])

test_acc = 0.0
model = Sequential()
interation = 0
while (
    test_acc < DNN_EXPLORATION_TARGET_VAL_ACCURACY
    and interation < DNN_EXPLORATION_MAX_ITER
):
    model = Sequential()
    model.add(InputLayer(input_shape=(n_features,)))
    model.add(BatchNormalization())
    dense_count = random.randint(
        DNN_EXPLORATION_HIDDEN_LAYERS_MIN, DNN_EXPLORATION_HIDDEN_LAYERS_MAX
    )
    dense_neurons = []
    for i in range(0, dense_count):
        neurons = random.randint(
            DNN_EXPLORATION_NEURONS_MIN, DNN_EXPLORATION_NEURONS_MAX
        )
        dense_neurons.append(neurons)
        model.add(Dense(neurons, activation="tanh"))
        model.add(BatchNormalization())
    model.add(Dense(categories, activation="sigmoid"))
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )
    model.fit(
        train_x,
        onehot_encoded_train_y,
        epochs=DNN_EPOCHS,
        batch_size=DNN_BATCH_SIZE,
        verbose=1,
        validation_data=(test_x, onehot_encoded_test_y),
        callbacks=[
            EarlyStopping(
                monitor="val_loss",
                patience=DNN_EARLY_STOPPING_PATIENCE,
                restore_best_weights=True,
            )
        ],
    )

    train_loss, train_acc = model.evaluate(train_x, onehot_encoded_train_y)
    test_loss, test_acc = model.evaluate(test_x, onehot_encoded_test_y)

    exploration_results.append(
        {
            "dense_count": dense_count,
            "dense_neurons": dense_neurons,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "test_loss": test_loss,
            "test_acc": test_acc,
        }
    )
    interation += 1

model.save(PATH_MODEL+"dnn."+str(round(time.time())))

Epoch 1/100
132/132 [==============================] - 1s 3ms/step - loss: 0.9004 - accuracy: 0.6606 - val_loss: 1.0207 - val_accuracy: 0.5953
Epoch 2/100
132/132 [==============================] - 0s 2ms/step - loss: 0.5678 - accuracy: 0.7988 - val_loss: 1.3973 - val_accuracy: 0.5450
Epoch 3/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4628 - accuracy: 0.8342 - val_loss: 2.0212 - val_accuracy: 0.5111
Epoch 4/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3901 - accuracy: 0.8565 - val_loss: 2.3167 - val_accuracy: 0.5017
Epoch 5/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3379 - accuracy: 0.8765 - val_loss: 2.3918 - val_accuracy: 0.5269
Epoch 6/100
132/132 [==============================] - 0s 2ms/step - loss: 0.3018 - accuracy: 0.8843 - val_loss: 2.7863 - val_accuracy: 0.5153
Epoch 7/100
132/132 [==============================] - 0s 2ms/step - loss: 0.2637 - accuracy: 0.9014 - val_loss: 2.8136 - val_accuracy: 0.5164

INFO:tensorflow:Assets written to: ./models/dnn.1685239363\assets


INFO:tensorflow:Assets written to: ./models/dnn.1685239363\assets


## 6. Modelanalyse des Learners [Kevin]

- Vorherige Auswahl eines Learners
- Feature Importance
- Korrelationsmatrix
- Konfusionsmatrix
- Post-Validation des Models mit auswählbaren Daten


## 7. Statische Interpretation des Resultats

- Welches Ergebnis haben wir erzielt und wie kann man es anwenden?
